 # Sugar tutorial





The objective of Sugar library is to provide a set of handy tools to facilitate the setup of simulations of polyelectrolytes and peptides with the Molecular Dynamics software ESPResSo. In other words: adding sugar to your ESPResSo scripts makes your life sweater. In this tutorial, the different functionalities of sugar are explained with different practical examples.

## Introduction

Let us get started by importing sugar library and creating an instance of it. When sugar is inicialitated, the default system of reduced units is defined:

In [1]:
import sugar
sg=sugar.sugar_library()

When sugar is inicialitated, a default system of reduced units is defined. The active set of reduced units can be consulted using

In [ ]:
sg.print_reduced_units()

This default definition of reduced units can be changed at the convience of the user using the command

In [ ]:
sg.set_reduced_units(unit_length=0.5*sg.units.nm,  unit_charge=2*sg.units.e, temperature=300 * sg.units.K)

All input variables will be given to ESPResSo using these reduced units, since it is a convinient choice for the simulation setup. Internally, Sugar uses Pint library to deal with unit transformations, which in turn should be  used by the user to define its own variables

In [2]:
Box_L = 5 * sg.units.nm

print('The side of the simulation box is ', Box_L, '=' ,Box_L.to('reduced_length'))

import espressomd

# Create an instance of an ESPResSo system 

system=espressomd.System(box_l=[Box_L.to('reduced_length').magnitude]*3)

The side of the simulation box is  5 nanometer = 14.084507042253522 reduced_length


## Use Sugar to create particles in ESPResSo

Particles are the smaller objects in the simulation box, which can represent small ions or other small chemical species. In turn, particles can also be used as building blocks for larger molecules or polymers, where they can represent one monomeric units. Particle objects are used in sugar as input for several of its funcionalities, including to create larger molecules and peptides. The basic properties of a particle are:

In [ ]:

cation=sg.particle(name='Na', 
                   type=0, 
                   q=1, 
                   diameter=0.35*sg.units.nm, 
                   epsilon=1*sg.units('reduced_energy'))

print('label for the particle', cation.name)
print('type of the particle to be identified within espresso', cation.type)
print('charge of the particle', cation.q)

# These parameters are only requiered for posterior setting up of Lennard-Jones interactions 

print('diameter of the particle', cation.diameter)
print('epsilon of the particle', cation.epsilon)



One can use sugar to create any number of the defined particles into the espresso system

In [ ]:
N_cations=20

for _ in range(N_cations):
    sg.create_object_in_system(object=cation, system=system)

Let us see the particle that we created by visualizing our espresso system

In [ ]:
picture_name='carbon_system.png'
sg.do_snapshot_system(system=system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
sg.destroy_object_in_system(object=cation, system=system)

## Use Sugar to create molecules in ESPResSo

Particles can be jointed together to form residues which in turn can be using as building blocks to construct molecules. A residue is composed by a `central_bead` where one or various `side_chains` are attached. Each side chain can contain one particle or other residues. For instance, to create the Chloroform molecule (CH<sub>3</sub>-CCl<sub>3</sub>)...

In [ ]:
# Define the different particle types in chloroform
# Use sg.propose_unused_type() to let sugar one non-used particle type for you

carbon=sg.particle(name='C', 
                   type=sg.propose_unused_type(),
                   q=0, 
                   diameter=0.4*sg.units.nm, 
                   epsilon=1*sg.units('reduced_energy'))

chlorine=sg.particle(name='Cl', 
                   type=sg.propose_unused_type(),
                   q=0, 
                   diameter=0.5*sg.units.nm, 
                   epsilon=1*sg.units('reduced_energy'))

hydrogen=sg.particle(name='H', 
                   type=sg.propose_unused_type(),
                   q=0, 
                   diameter=0.3*sg.units.nm, 
                   epsilon=1*sg.units('reduced_energy'))

# Split chloroform in two residues

methylene=sg.residue(name='met',
                    central_bead=carbon,
                    side_chains=[hydrogen,hydrogen,hydrogen])

tricloromethylene=sg.residue(name='Clmet',
                    central_bead=carbon,
                    side_chains=[chlorine]*3)

# Define the chloroform molecule as the combination of the two residues

chloroform=sg.molecule(name='',residue_list=[methylene,tricloromethylene])

# Define the bonds between the particles

from espressomd import interactions

generic_bond_lenght=0.4 * sg.units.nm
generic_harmonic_constant=400*sg.units('reduced_energy / nm**2') 
generic_bond = interactions.HarmonicBond(k=generic_harmonic_constant.to('reduced_energy / reduced_length**2').magnitude,
                                 r_0=generic_bond_lenght.to('reduced_length').magnitude)

sg.define_bond(bond=generic_bond, particle1=carbon, particle2=carbon)
sg.define_bond(bond=generic_bond, particle1=carbon, particle2=hydrogen)
sg.define_bond(bond=generic_bond, particle1=carbon, particle2=chlorine)

sg.add_bonds_to_system(system=system)

# Create a chloroform molecule into the espresso system

N_chloroform=1

for _ in range(N_chloroform):
    sg.create_object_in_system(object=chloroform, system=system, position=[Box_L.to('reduced_length').magnitude/2]*3)

# Do a small thermalization of the system to get a nicer picture

sg.setup_lj_interactions(system=system)
sg.minimize_system_energy(system=system)
    
picture_name='chloroform_system.png'
sg.do_snapshot_system(system=system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
sg.destroy_object_in_system(object=chloroform, system=system)


## Use Sugar to create polymers in ESPResSo

Sugar can be used also to set up fastly polymers with different arquitechtures (e.g. alternate polymer, di-block polymers...). For instance, we can consider one model case of a di-block polymer [C(CH<sub>3</sub>)]<sub>5</sub>-[C(CCl<sub>3</sub>)]<sub>5</sub>

In [ ]:
# Define the backbone residues

backbone_CH3=sg.residue(name='back-CH3',
                    central_bead=carbon,
                    side_chains=[methylene])

backbone_Cl=sg.residue(name='back-CCl3',
                    central_bead=carbon,
                    side_chains=[tricloromethylene])

N_units=4

di_block_polymer=sg.molecule(name='block_pol',residue_list=[backbone_CH3]*N_units+[backbone_Cl]*N_units)

# Create a chloroform molecule into the espresso system

N_pol=1

for _ in range(N_pol):
    sg.create_object_in_system(object=di_block_polymer, system=system, position=[Box_L.to('reduced_length').magnitude/2]*3)

# Do a small thermalization of the system to get a nicer picture

sg.minimize_system_energy(system=system)
    
picture_name='diblock_system.png'
sg.do_snapshot_system(system=system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
sg.destroy_object_in_system(object=di_block_polymer, system=system)

One can easilly set up instead the alternating case of our model polymer  [C(CH<sub>3</sub>)-C(CCl<sub>3</sub>)]<sub>5</sub> using sugar

In [ ]:
alternating_polymer=sg.molecule(name='alt_pol',residue_list=[backbone_CH3,backbone_Cl]*N_units)

# Create a chloroform molecule into the espresso system

N_pol=1

for _ in range(N_pol):
    sg.create_object_in_system(object=alternating_polymer, system=system, position=[Box_L.to('reduced_length').magnitude/2]*3)

# Do a small thermalization of the system to get a nicer picture

sg.minimize_system_energy(system=system)
    
picture_name='diblock_system.png'
sg.do_snapshot_system(system=system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
sg.destroy_object_in_system(object=alternating_polymer, system=system)

## Use Sugar to create peptides 

Sugar includes built-on functions to facilitate the setting up of coarse-grained models for peptides from their aminoacid sequence. At the moment, two different coarse-grained models have been implemented with one bead (`model=1beadAA`) or two beads (`model=2beadAA`) per aminoacid. For the aminoacids currently available, we provide reference parameters in the folder (`reference_parameters`) which can be loaded into Sugar

In [3]:
N_aminoacids=5
N_peptide=1
sequence="K"*N_aminoacids+"D"*N_aminoacids
model='2beadAA'  # Model with 2 beads per each aminoacid

 # Load peptide parametrization from Lunkad, R. et al.  Molecular Systems Design & Engineering (2021), 6(2), 122-131.

sg.load_parameters(filename='reference_parameters/Lunkad2021.txt')
sg.add_bonds_to_system(system=system) # Add the bonds loaded from the file to the espresso system

peptide = sg.peptide(name=sequence, sequence=sequence, model=model)

for _ in range(N_peptide):
    sg.create_object_in_system(object=peptide, system=system, position=[Box_L.to('reduced_length').magnitude/2]*3)

# Do a small thermalization of the system to get a nicer picture

sg.minimize_system_energy(system=system)
    
picture_name='peptide_system.png'
sg.do_snapshot_system(system=system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
sg.destroy_object_in_system(object=peptide, system=system)


*** Minimazing system energy... ***

steepest descent
velocity verlet

 Minimization finished 



Note that in `sequence` we have used the one letter code for the aminoacids. Internally, Sugar uses the one-letter code to define each aminoacid. The three letter code can also be used as input, which Sugar automatically detects and transforms into the one-letter code using its own protein sequence parser.

In [4]:
# The same peptide as before, but now using the three letter code for aminoacids

sequence='LYS-LYS-LYS-LYS-LYS-GLU-GLU-GLU-GLU-GLU'
peptide = sg.peptide(name=sequence, sequence=sequence, model=model)
print('one letter code', sg.protein_sequence_parser(sequence=sequence))
print('defined peptide sequence ', peptide.sequence)


one letter code ['K', 'K', 'K', 'K', 'K', 'E', 'E', 'E', 'E', 'E']
defined peptide sequence  ['K', 'K', 'K', 'K', 'K', 'E', 'E', 'E', 'E', 'E']


In [ ]:
molecule_list=[cation,
               anion, 
               strong_polyampholyte,
               weak_polyampholyte,
               one_bead_peptide,
               two_bead_peptide,
               custom_two_bead_peptide]

sg.setup_lj_interactions(mol_list=molecule_list, system=system)

You can also setup electrostatic interactions in your system using Sugar. By default, p3m method is used but the Debye-Huckel potential is also implemented

In [ ]:
method='DH'

sg.setup_electrostatic_interactions(system=system, c_salt=c_salt, method=method)

Once all you have added all your interactions to your system, you can use sugar to minimize your system energy

In [ ]:
sg.minimize_system_energy(system=system)

and to setup the thermostat for Langevin Dynamics

In [ ]:
# By default time_step=1e-2, it is decreased here for enhaced visualization
sg.setup_langevin_dynamics(system=system, time_step=5e-4) 

Note that in these processes many parameters are setup to default values that are reasonable choices in general but that should always be checked for your specific system. Please, refer to the documentation of each function for further details. Espresso visualizer can be used to check the molecules that we have created throught this tutorial

In [ ]:
from espressomd import visualization

visualizer = visualization.openGLLive(system, bond_type_radius=[0.3])

system.integrator.run(steps=1000) # to equilibrate the system

visualizer.run()

### References

[1] Hass MA, Mulder FAA. Contemporary NMR Studies of Protein Electrostatics. Annu Rev Biophys. 2015;44:53-75.

[2] Platzer G, Okon M, McIntosh LP. 2014. pH-dependent random coil 1 H, 13 C, and 15 N chemical shifts of the ionizable amino acids: a guide for protein pK a measurements. J. Biomol. NMR 60:109–29

[3] Handbook of Chemistry and Physics, 72nd Edition, CRC Press, Boca Raton, FL, 1991.

[4] Y. Nozaki and C. Tanford, Methods Enzymol., 1967, 11, 715–734.

[5] Bienkiewicz & K.J. Lumb, J Biomol NMR 15: 203-206 (1999).